# BERT's playground 
Hello there! Welcome on BERT's playground. You may play with BERT here and see what he can do but always make sure he feels respected and admired.

## Setups


In [1]:
import sys
sys.path.append('../ml/BERT')
import masking
import BERT
from Vectorisation import Vectorisation
from Config import Config
from MaskedTextGenerator import MaskedTextGenerator

import pickle
import tensorflow as tf
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split

with open("../../data/ml4science_data.pkl", "rb") as fp:
    data_dict = pickle.load(fp)

config = Config()
vec = Vectorisation(config=config)

# Testing BERT

In [2]:
# Prepare data for masked language model
encoded = vec.encode_dict(data_dict)
encoded_train, encoded_test = train_test_split(encoded, test_size=0.2, random_state=42)
x_masked_encoded, y_masked_encoded, sample_weights = masking.mask_input_and_labels(encoded_train, config.TOKEN_DICT, seed=32)
x_test, y_test, _ = masking.mask_input_and_labels(encoded_test, config.TOKEN_DICT, seed=32)
print(x_masked_encoded.shape, y_masked_encoded.shape, sample_weights.shape)
print(x_test.shape, y_test.shape)

mlm_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded, sample_weights))
mlm_ds = mlm_ds.shuffle(1000).batch(config.BATCH_SIZE)

print(mlm_ds)

(203, 128) (203, 128) (203, 128)
(51, 128) (51, 128)
<_BatchDataset element_spec=(TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.int32, name=None), TensorSpec(shape=(None, 128), dtype=tf.float64, name=None))>


In [3]:
encoded

array([[26,  2, 26, ...,  0,  0,  0],
       [26, 21, 26, ..., 14, 11, 14],
       [25, 26, 21, ...,  0,  0,  0],
       ...,
       [26,  2, 26, ...,  0,  0,  0],
       [26, 25, 26, ...,  0,  0,  0],
       [26,  2, 26, ..., 22, 26, 21]])

In [4]:
sample_tokens = x_masked_encoded[0:1]
print(y_masked_encoded[0])

[26  2 26 21 26 21 26 23 26 21 26 23 26 21 26 21 26 21 26 21 26 21 26 21
 26  2 26 23 26 21 26  9 14 11 14  9 14 13 14 10 14 10 14 10 14 10 14 10
 14 10 14 10 14 10 14 10 14 10 14 10 14 10 14 10 14 10 14 10 10 10 10 10
 14 10 10 10 10 10 14 10 10 10 10 10 10 10 10 10 10 10 10 10 14 11 14  9
 14  9 14 10 14  2 14  9 14 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 14 10]


In [5]:
print(x_masked_encoded[0])

[26  2 26 21 26 21  1 23 26 21  1 23 26 21 26 21 26  1 26 21 26  1 26 21
 26  2 26 23 26 21  1  9 14 11  1  9 14 13 14 10 14 10 14 10 14 10 14 10
 14  1 14 10 14  1 14 10 14 10 14 10  1 10 14  1  1 10 14 10 10 10 10  1
 14 10 10 10 10 10 14 10 10 10 10 10 10  1 10 10  1 10 10 10 14 11 14  9
 14  9 14 10 14  2 14  9 14 10 10 10  1  1 10 10 10 10 10 10 10 10 10 10
 10 10  1 10 10 10 14 10]


In [6]:
generator_callback = MaskedTextGenerator(sample_tokens, config.TOKEN_DICT['[MASK]'])

bert_masked_model = BERT.create_masked_language_bert_model(config)
bert_masked_model.summary()


Model: "masked_bert_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 word_embedding (Embedding)  (None, 128, 64)              1728      ['input_1[0][0]']             
                                                                                                  
 tf.__operators__.add (TFOp  (None, 128, 64)              0         ['word_embedding[0][0]']      
 Lambda)                                                                                          
                                                                                                  
 encoder_0/multiheadattenti  (None, 128, 64)              16640     ['tf.__operat

In [7]:
# A callback in Keras is a function that is called at certain points during the training process. -> Here is called after each epoch during the training
# Here we use to see the "performance" at each epoch while predicting on a "test set" aka the sample_tokens

bert_masked_model.fit(mlm_ds, epochs=100)#, callbacks=[generator_callback]) 

Epoch 1/100


13/13 [==============================] - 2s 11ms/step - loss: 3.2831
Epoch 2/100
13/13 [==============================] - 0s 11ms/step - loss: 2.7629
Epoch 3/100
13/13 [==============================] - 0s 12ms/step - loss: 2.5998
Epoch 4/100
13/13 [==============================] - 0s 11ms/step - loss: 2.5309
Epoch 5/100
13/13 [==============================] - 0s 10ms/step - loss: 2.4706
Epoch 6/100
13/13 [==============================] - 0s 10ms/step - loss: 2.4478
Epoch 7/100
13/13 [==============================] - 0s 10ms/step - loss: 2.4206
Epoch 8/100
13/13 [==============================] - 0s 10ms/step - loss: 2.4049
Epoch 9/100
13/13 [==============================] - 0s 10ms/step - loss: 2.3730
Epoch 10/100
13/13 [==============================] - 0s 10ms/step - loss: 2.3281
Epoch 11/100
13/13 [==============================] - 0s 10ms/step - loss: 2.2749
Epoch 12/100
13/13 [==============================] - 0s 10ms/step - loss: 2.1975
Epoch 13/100
13/13 [=================

In [9]:
predictions = bert_masked_model.predict(x_masked_encoded)

predictions_max = np.argmax(predictions, axis=2)
print("Predictions:\n",predictions_max[0])
print("Original:\n", y_masked_encoded[0])
print("Masked:\n", x_masked_encoded[0])

7/7 [==============================] - 0s 8ms/step
Predictions:
 [26 26 26 21 26 21 26 26 26 21 26 23 26 21 26 21 26 26 26 21 26 21 26 21
 26 26 26 26 26 21 26  9 14 14 14 14 14 13 14 10 14 10 14 10 14 10 14 10
 14 10 14 10 14 10 14 10 14 10 14 10 10 10 14 10 10 10 14 10 10 10 10 10
 14 10 10 10 10 10 14 10 10 10 10 10 10 10 10 10 10 10 10 10 14 11 14 10
 14 10 14 10 14 10 14  9 14 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 14 10]
Original:
 [26  2 26 21 26 21 26 23 26 21 26 23 26 21 26 21 26 21 26 21 26 21 26 21
 26  2 26 23 26 21 26  9 14 11 14  9 14 13 14 10 14 10 14 10 14 10 14 10
 14 10 14 10 14 10 14 10 14 10 14 10 14 10 14 10 14 10 14 10 10 10 10 10
 14 10 10 10 10 10 14 10 10 10 10 10 10 10 10 10 10 10 10 10 14 11 14  9
 14  9 14 10 14  2 14  9 14 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10
 10 10 10 10 10 10 14 10]
Masked:
 [26  2 26 21 26 21  1 23 26 21  1 23 26 21 26 21 26  1 26 21 26  1 26 21
 26  2 26 23 26 21  1  9 14 11  1  9 14 13 14 10 14 10 14 10

In [10]:
print(predictions_max[0] == y_masked_encoded[0])
# print accuracy
print("Accuracy brut: ", np.sum(predictions_max == y_masked_encoded) / (100*len(y_masked_encoded)))
# print accuracy without padding
print("Accuracy without taking in acount padding: ", np.sum((predictions_max == y_masked_encoded) * (y_masked_encoded!= 0)) / np.sum(y_masked_encoded != 0))
print("Accuracy on masked tokens: ", np.sum((predictions_max == y_masked_encoded) * (x_masked_encoded == config.TOKEN_DICT['[MASK]'])) / np.sum(x_masked_encoded == config.TOKEN_DICT['[MASK]']))

[ True False  True  True  True  True  True False  True  True  True  True
  True  True  True  True  True False  True  True  True  True  True  True
  True False  True False  True  True  True  True  True False  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True False  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True False
  True False  True  True  True False  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True]
Accuracy brut:  0.6125615763546798
Accuracy without taking in acount padding:  0.8138621637541724
Accuracy on masked tokens:  0.7917647058823529


In [11]:
predictions = bert_masked_model.predict(x_test)

predictions_max = np.argmax(predictions, axis=2)
print("Predictions:\n",predictions_max[0])
print("Original:\n", y_test[0])
print("Masked:\n", x_test[0])

2/2 [==============================] - 0s 8ms/step
Predictions:
 [26 26 26 21 26 26 26 26 26 21 26 26 26 26 23 26 26 26 21 26 21 26 21 26
 21 26 21 26 26 26 26 22 22 22 21 22 22 22 22 22 22 22 22 22 22 22 22 22
 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22
 22 22 22 22 22 22 22 22 22 22 22 22 22 26 26 22 22 22 26 23 26 22 22 22
 22 22 22 22 22 22 22 22 22 22 22 22 22 22 26 22 21  4  4 22 22 22 21 22
 21  8 21  8  8  8  8  8]
Original:
 [26  2 26 21 26 21 26 21 26 21 23 26 22 26 23 26  2 26 21 26 21 26 21 26
  2 26 21 26 22 26 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22
 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22 22
 22 22 22 22 22 22 22 22 22 22 22 22 22 22 26 23 26 23 26 23 26 22 22 22
 22 22 22 22 22 22 26  3  8  5  8  4  8 21 26  3  8  4  4  8  3  8  5  8
  5  8  5  8  4  8  3  8]
Masked:
 [26  2 26 21 26 21  1 21 26 21  1 26 22 26 23 26  2  1 21 26 21  1 21 26
  2 26 21 26 22 26  1 22 22 22 21 22 22 22 22 22 22 22 22 22

In [12]:
print(predictions_max[0] == y_test[0])
# print accuracy
print("Accuracy brut: ", np.sum(predictions_max == y_test) / (100*len(y_test)))
# print accuracy without padding
print("Accuracy without taking in acount padding: ", np.sum((predictions_max == y_test) * (y_test!= 0)) / np.sum(y_test != 0))
print("Accuracy on masked tokens: ", np.sum((predictions_max == y_test) * (x_test == config.TOKEN_DICT['[MASK]'])) / np.sum(x_test == config.TOKEN_DICT['[MASK]']))

[ True False  True  True  True False  True False  True  True False  True
 False  True  True  True False  True  True  True  True  True  True  True
 False  True  True  True False  True False  True  True  True False  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True False False False  True  True  True  True  True  True
  True  True  True  True  True  True False False False False False False
 False False  True False False  True  True False False False False False
 False  True False  True False  True False  True]
Accuracy brut:  0.49470588235294116
Accuracy without taking in acount padding:  0.7285590528443546
Accuracy on masked tokens:  0.34782608695652173


In [10]:
# This how we can load a Keras model
"""# Load OUR pretrained bert model
mlm_model = keras.models.load_model(
    "bert_mlm_imdb.keras", custom_objects={"MaskedLanguageModel": MaskedLanguageModel}
)"""

# Kinda failed attempt to create a end to end model (we don't really need it) -> But it's a good example of how to create a model with a custom loss function and reshape the output
"""def get_end_to_end(model):
    inputs = keras.Input(shape=(None,))
    outputs = model(inputs)
    reshaped_outputs = keras.layers.Lambda(lambda x: keras.backend.argmax(x, axis=-1))(outputs)
    end_to_end_model = keras.Model(inputs, reshaped_outputs, name="end_to_end_model")
    optimizer = keras.optimizers.Adam(learning_rate=config.bert.LR)
    end_to_end_model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )
    return end_to_end_model

end_to_end_classification_model = get_end_to_end(bert_masked_model)

# Build dataset for end to end model input (will be used at the end)
test_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded)).batch(config.BATCH_SIZE)

end_to_end_classification_model.evaluate(test_raw_classifier_ds)"""

'def get_end_to_end(model):\n    inputs = keras.Input(shape=(None,))\n    outputs = model(inputs)\n    reshaped_outputs = keras.layers.Lambda(lambda x: keras.backend.argmax(x, axis=-1))(outputs)\n    end_to_end_model = keras.Model(inputs, reshaped_outputs, name="end_to_end_model")\n    optimizer = keras.optimizers.Adam(learning_rate=config.bert.LR)\n    end_to_end_model.compile(\n        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]\n    )\n    return end_to_end_model\n\nend_to_end_classification_model = get_end_to_end(bert_masked_model)\n\n# Build dataset for end to end model input (will be used at the end)\ntest_raw_classifier_ds = tf.data.Dataset.from_tensor_slices((x_masked_encoded, y_masked_encoded)).batch(config.BATCH_SIZE)\n\nend_to_end_classification_model.evaluate(test_raw_classifier_ds)'